In [556]:
from __future__ import print_function

from functools import partial
from urllib.request import urlretrieve
import pandas as pd
import torch
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler



# hyper-opt
from hyperopt import hp, Trials, STATUS_OK, tpe, fmin
from hyperopt import space_eval
from hyperopt.pyll import scope

from tensorflow.keras.layers import Dense, Dropout, Activation
from tensorflow.keras.models import Sequential

# utils
import pickle
import argparse
import os
import numpy as np 
from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import cross_val_score, cross_validate
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import AdaBoostClassifier
import xgboost
from sklearn.metrics import accuracy_score, precision_score, recall_score

In [557]:
urlretrieve('http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data', 'german.data')

('german.data', <http.client.HTTPMessage at 0x7f4cd2f6d310>)

In [558]:
german_df = pd.read_csv('german.data', delimiter=' ',header=None)

In [559]:
german_df.head()


,0,1,2,3,4,5,6,7,8,9,...,11,12,13,14,15,16,17,18,19,20
0,A11,6,A34,A43,1169,A65,A75,4,A93,A101,...,A121,67,A143,A152,2,A173,1,A192,A201,1
1,A12,48,A32,A43,5951,A61,A73,2,A92,A101,...,A121,22,A143,A152,1,A173,1,A191,A201,2
2,A14,12,A34,A46,2096,A61,A74,2,A93,A101,...,A121,49,A143,A152,1,A172,2,A191,A201,1
3,A11,42,A32,A42,7882,A61,A74,2,A93,A103,...,A122,45,A143,A153,1,A173,2,A191,A201,1
4,A11,24,A33,A40,4870,A61,A73,3,A93,A101,...,A124,53,A143,A153,2,A173,2,A191,A201,2


In [560]:
urlretrieve('http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc', 'german.doc')
f = open('german.doc')
german_doc= f.read()
# print(german_doc)

In [561]:
german_df.columns=['account_bal','duration','payment_status','purpose',
                   'credit_amount','savings_bond_value','employed_since',
                   'intallment_rate','sex_marital','guarantor','residence_since',
                   'most_valuable_asset','age','concurrent_credits','type_of_housing',
                   'number_of_existcr','job','number_of_dependents','telephon',
                   'foreign','target']

In [562]:
german_df= german_df.replace(['A11','A12','A13','A14', 'A171','A172','A173','A174','A121','A122','A123','A124'],
                  ['neg_bal','positive_bal','positive_bal','no_acc','unskilled','unskilled','skilled','highly_skilled',
                   'none','car','life_insurance','real_estate'])

In [563]:
german_df.head()

,account_bal,duration,payment_status,purpose,credit_amount,savings_bond_value,employed_since,intallment_rate,sex_marital,guarantor,...,most_valuable_asset,age,concurrent_credits,type_of_housing,number_of_existcr,job,number_of_dependents,telephon,foreign,target
0,neg_bal,6,A34,A43,1169,A65,A75,4,A93,A101,...,none,67,A143,A152,2,skilled,1,A192,A201,1
1,positive_bal,48,A32,A43,5951,A61,A73,2,A92,A101,...,none,22,A143,A152,1,skilled,1,A191,A201,2
2,no_acc,12,A34,A46,2096,A61,A74,2,A93,A101,...,none,49,A143,A152,1,unskilled,2,A191,A201,1
3,neg_bal,42,A32,A42,7882,A61,A74,2,A93,A103,...,car,45,A143,A153,1,skilled,2,A191,A201,1
4,neg_bal,24,A33,A40,4870,A61,A73,3,A93,A101,...,real_estate,53,A143,A153,2,skilled,2,A191,A201,2


In [564]:
import numpy as np
import torch

In [565]:
# Number of unique classes in each object column
german_df.select_dtypes('object').apply(pd.Series.nunique, axis = 0)

account_bal             3
payment_status          5
purpose                10
savings_bond_value      5
employed_since          5
sex_marital             4
guarantor               3
most_valuable_asset     4
concurrent_credits      3
type_of_housing         3
job                     3
telephon                2
foreign                 2
dtype: int64

In [566]:
german_df['age']

0      67
1      22
2      49
3      45
4      53
       ..
995    31
996    40
997    38
998    23
999    27
Name: age, Length: 1000, dtype: int64

In [567]:
from sklearn.preprocessing import LabelEncoder

le= LabelEncoder()
le.fit(german_df.target)
german_df.target=le.transform(german_df.target)
german_df.target.head(5)

0    0
1    1
2    0
3    0
4    1
Name: target, dtype: int64

In [568]:
german_df

,account_bal,duration,payment_status,purpose,credit_amount,savings_bond_value,employed_since,intallment_rate,sex_marital,guarantor,...,most_valuable_asset,age,concurrent_credits,type_of_housing,number_of_existcr,job,number_of_dependents,telephon,foreign,target
0,neg_bal,6,A34,A43,1169,A65,A75,4,A93,A101,...,none,67,A143,A152,2,skilled,1,A192,A201,0
1,positive_bal,48,A32,A43,5951,A61,A73,2,A92,A101,...,none,22,A143,A152,1,skilled,1,A191,A201,1
2,no_acc,12,A34,A46,2096,A61,A74,2,A93,A101,...,none,49,A143,A152,1,unskilled,2,A191,A201,0
3,neg_bal,42,A32,A42,7882,A61,A74,2,A93,A103,...,car,45,A143,A153,1,skilled,2,A191,A201,0
4,neg_bal,24,A33,A40,4870,A61,A73,3,A93,A101,...,real_estate,53,A143,A153,2,skilled,2,A191,A201,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,no_acc,12,A32,A42,1736,A61,A74,3,A92,A101,...,none,31,A143,A152,1,unskilled,1,A191,A201,0
996,neg_bal,30,A32,A41,3857,A61,A73,4,A91,A101,...,car,40,A143,A152,1,highly_skilled,1,A192,A201,0
997,no_acc,12,A32,A43,804,A61,A75,4,A93,A101,...,life_insurance,38,A143,A152,1,skilled,1,A191,A201,0
998,neg_bal,45,A32,A43,1845,A61,A73,4,A93,A101,...,real_estate,23,A143,A153,1,skilled,1,A192,A201,1


In [569]:
# sklearn preprocessing for dealing with categorical variables
from sklearn.preprocessing import LabelEncoder

# Create a label encoder object
le = LabelEncoder()
le_count = 0

# Iterate through the columns
for col in german_df:
    if german_df[col].dtype == 'object':
        # If 2 or fewer unique categories
        if len(list(german_df[col].unique())) <= 2:
            # Train on the training data
            le.fit(german_df[col])
            # Transform both training and testing data
            german_df[col] = le.transform(german_df[col])
            
            # Keep track of how many columns were label encoded
            le_count += 1
            
print('%d columns were label encoded.' % le_count)

2 columns were label encoded.


In [570]:
# one-hot encoding of categorical variables
german_df = pd.get_dummies(german_df)

print('Encoded Features shape: ', german_df.shape)

Encoded Features shape:  (1000, 58)


In [571]:
german_df.head()

,duration,credit_amount,intallment_rate,residence_since,age,number_of_existcr,number_of_dependents,telephon,foreign,target,...,most_valuable_asset_real_estate,concurrent_credits_A141,concurrent_credits_A142,concurrent_credits_A143,type_of_housing_A151,type_of_housing_A152,type_of_housing_A153,job_highly_skilled,job_skilled,job_unskilled
0,6,1169,4,4,67,2,1,1,0,0,...,0,0,0,1,0,1,0,0,1,0
1,48,5951,2,2,22,1,1,0,0,1,...,0,0,0,1,0,1,0,0,1,0
2,12,2096,2,3,49,1,2,0,0,0,...,0,0,0,1,0,1,0,0,0,1
3,42,7882,2,4,45,1,2,0,0,0,...,0,0,0,1,0,0,1,0,1,0
4,24,4870,3,4,53,2,2,0,0,1,...,1,0,0,1,0,0,1,0,1,0


In [572]:
# german_df= german_df.replace({'age': {'<=50K': 0, '<=50K.': 0,  '>50K': 1, '>50K.': 1}})
german_df['age_age:<25'] = 0
german_df['age_age:>=25'] = 0
for i in range(german_df['age'].shape[0]):
    print('i = ',  i)
    if  german_df['age'][i] < 25 :
        german_df['age_age:<25'][i] = '1'
        german_df['age_age:>=25'][i] = '0'
    else:
        german_df['age_age:<25'][i] = '0'
        german_df['age_age:>=25'][i] = '1'
    
    

german_df.drop('age', axis = 1)

i =  0
i =  1
i =  2
i =  3
i =  4
i =  5
i =  6
i =  7
i =  8
i =  9
i =  10
i =  11
i =  12
i =  13
i =  14
i =  15
i =  16
i =  17
i =  18
i =  19
i =  20
i =  21
i =  22
i =  23
i =  24
i =  25
i =  26
i =  27
i =  28
i =  29
i =  30
i =  31
i =  32
i =  33
i =  34
i =  35
i =  36
i =  37
i =  38
i =  39
i =  40
i =  41
i =  42
i =  43
i =  44
i =  45
i =  46
i =  47
i =  48
i =  49
i =  50
i =  51
i =  52
i =  53
i =  54
i =  55
i =  56
i =  57
i =  58
i =  59
i =  60
i =  61
i =  62
i =  63
i =  64
i =  65
i =  66
i =  67
i =  68
i =  69
i =  70
i =  71
i =  72
i =  73
i =  74
i =  75
i =  76
i =  77
i =  78
i =  79
i =  80
i =  81
i =  82
i =  83
i =  84
i =  85
i =  86
i =  87
i =  88
i =  89
i =  90
i =  91
i =  92
i =  93
i =  94
i =  95
i =  96
i =  97
i =  98
i =  99
i =  100
i =  101
i =  102
i =  103
i =  104
i =  105
i =  106
i =  107
i =  108
i =  109
i =  110
i =  111
i =  112
i =  113
i =  114
i =  115
i =  116
i =  117
i =  118
i =  119
i =  120
i =  121
i =  122
i =

/pkgs/jupyterhub/lib/python3.7/site-packages/ipykernel_launcher.py:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # Remove the CWD from sys.path while we load stuff.
/pkgs/jupyterhub/lib/python3.7/site-packages/ipykernel_launcher.py:11: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  # This is added back by InteractiveShellApp.init_path()
/pkgs/jupyterhub/lib/python3.7/site-packages/ipykernel_launcher.py:7: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-vi


i =  182
i =  183
i =  184
i =  185
i =  186
i =  187
i =  188
i =  189
i =  190
i =  191
i =  192
i =  193
i =  194
i =  195
i =  196
i =  197
i =  198
i =  199
i =  200
i =  201
i =  202
i =  203
i =  204
i =  205
i =  206
i =  207
i =  208
i =  209
i =  210
i =  211
i =  212
i =  213
i =  214
i =  215
i =  216
i =  217
i =  218
i =  219
i =  220
i =  221
i =  222
i =  223
i =  224
i =  225
i =  226
i =  227
i =  228
i =  229
i =  230
i =  231
i =  232
i =  233
i =  234
i =  235
i =  236
i =  237
i =  238
i =  239
i =  240
i =  241
i =  242
i =  243
i =  244
i =  245
i =  246
i =  247
i =  248
i =  249
i =  250
i =  251
i =  252
i =  253
i =  254
i =  255
i =  256
i =  257
i =  258
i =  259
i =  260
i =  261
i =  262
i =  263
i =  264
i =  265
i =  266
i =  267
i =  268
i =  269
i =  270
i =  271
i =  272
i =  273
i =  274
i =  275
i =  276
i =  277
i =  278
i =  279
i =  280
i =  281
i =  282
i =  283
i =  284
i =  285
i =  286
i =  287
i =  288
i =  289
i =  290
i =  291
i =  292


,duration,credit_amount,intallment_rate,residence_since,number_of_existcr,number_of_dependents,telephon,foreign,target,account_bal_neg_bal,...,concurrent_credits_A142,concurrent_credits_A143,type_of_housing_A151,type_of_housing_A152,type_of_housing_A153,job_highly_skilled,job_skilled,job_unskilled,age_age:<25,age_age:>=25
0,6,1169,4,4,2,1,1,0,0,1,...,0,1,0,1,0,0,1,0,0,1
1,48,5951,2,2,1,1,0,0,1,0,...,0,1,0,1,0,0,1,0,1,0
2,12,2096,2,3,1,2,0,0,0,0,...,0,1,0,1,0,0,0,1,0,1
3,42,7882,2,4,1,2,0,0,0,1,...,0,1,0,0,1,0,1,0,0,1
4,24,4870,3,4,2,2,0,0,1,1,...,0,1,0,0,1,0,1,0,0,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
995,12,1736,3,4,1,1,0,0,0,0,...,0,1,0,1,0,0,0,1,0,1
996,30,3857,4,4,1,1,1,0,0,1,...,0,1,0,1,0,1,0,0,0,1
997,12,804,4,4,1,1,0,0,0,0,...,0,1,0,1,0,0,1,0,0,1
998,45,1845,4,4,1,1,1,0,1,1,...,0,1,0,0,1,0,1,0,1,0


In [580]:
# german_df['age'] = df['age']  
german_df[:20]

,duration,credit_amount,intallment_rate,residence_since,age,number_of_existcr,number_of_dependents,telephon,foreign,target,...,concurrent_credits_A142,concurrent_credits_A143,type_of_housing_A151,type_of_housing_A152,type_of_housing_A153,job_highly_skilled,job_skilled,job_unskilled,age_age:<25,age_age:>=25
0,6,1169,4,4,67,2,1,1,0,0,...,0,1,0,1,0,0,1,0,0,1
1,48,5951,2,2,22,1,1,0,0,1,...,0,1,0,1,0,0,1,0,1,0
2,12,2096,2,3,49,1,2,0,0,0,...,0,1,0,1,0,0,0,1,0,1
3,42,7882,2,4,45,1,2,0,0,0,...,0,1,0,0,1,0,1,0,0,1
4,24,4870,3,4,53,2,2,0,0,1,...,0,1,0,0,1,0,1,0,0,1
5,36,9055,2,4,35,1,2,1,0,0,...,0,1,0,0,1,0,0,1,0,1
6,24,2835,3,4,53,1,1,0,0,0,...,0,1,0,1,0,0,1,0,0,1
7,36,6948,2,2,35,1,1,1,0,0,...,0,1,1,0,0,1,0,0,0,1
8,12,3059,2,4,61,1,1,0,0,0,...,0,1,0,1,0,0,0,1,0,1
9,30,5234,4,2,28,2,1,0,0,1,...,0,1,0,1,0,1,0,0,0,1


In [581]:
x, y = german_df.drop('target', axis=1), german_df['target']
# x = x.to_numpy()
# y = y.to_numpy()
# x.shape, y.shape

In [582]:
x.shape

(1000, 59)

In [583]:
x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=.2, random_state=42)
# train, test= train_test_split(german_df, test_size=.2, random_state=42)

In [584]:
# np.save(f'german_credit_train.npy', {'X': x_train, 'y': y_train})
# np.save(f'german_credit_test.npy', {'X': x_test, 'y': y_test})
x_train.to_csv('train.csv')
x_test.to_csv('test.csv')
df_train    = pd.read_csv('train.csv')
df_test     = pd.read_csv('test.csv')

In [586]:
subgroup_dict = {
    'german_credit'     : ('age_age:>=25', 'age_age:<25'),
    'adult_income'      : ('gender_Female', 'gender_Male'),
    'compas'            : ('race_African-American', 'race_Caucasian'),
    'default_credit'    : ('SEX_Female', 'SEX_Male'),
    'marketing'         : ('age_age:not30-60', 'age_age:30-60')      
}

min_feature, maj_feature = subgroup_dict['german_credit']

In [589]:
german_df[min_feature]

0      1
1      0
2      1
3      1
4      1
      ..
995    1
996    1
997    1
998    0
999    1
Name: age_age:>=25, Length: 1000, dtype: int64

In [578]:
x_train = x_train.to_numpy()
y_train = y_train.to_numpy()
x_test = x_test.to_numpy()
y_test = y_test.to_numpy()

array([[  60, 6836,    3, ...,    0,    0,    1],
       [  21, 2319,    2, ...,    0,    0,    1],
       [   6, 1236,    2, ...,    0,    0,    1],
       ...,
       [  24, 5804,    4, ...,    0,    0,    1],
       [  12, 1484,    2, ...,    0,    0,    1],
       [   6,  932,    3, ...,    0,    1,    0]])

In [181]:
Batch_size = 100
X_train = torch.from_numpy(x_train) #2d [[],[],...]
y_train = torch.from_numpy(y_train) #1d [1,1,0,0,....]

#Valid 
X_Test = torch.from_numpy(x_test) 
y_Test = torch.from_numpy(y_test) 

import torch.utils.data as data
class AdultDataset(data.Dataset):

    def __init__(self,X, y): #X and Y 

        #pass
        ######
        # 4.1 YOUR CODE HERE
        self.X = X
        self.y = y
        ######

    def __len__(self):
        
        return len(self.X)

    def __getitem__(self, index): #get the (i)th sample 
        #pass
        
        ######
        # 4.1 YOUR CODE HERE
        item = self.X[index,:],self.y[index] #extract (i)th sample row and label for that sample
                
        return item 
        
        ######

In [182]:
X_train

tensor([[  60, 6836,    3,  ...,    0,    1,    0],
        [  21, 2319,    2,  ...,    0,    1,    0],
        [   6, 1236,    2,  ...,    0,    1,    0],
        ...,
        [  24, 5804,    4,  ...,    0,    1,    0],
        [  12, 1484,    2,  ...,    0,    1,    0],
        [   6,  932,    3,  ...,    0,    1,    0]])

In [191]:
X_train

array([[   6, 1169,    4, ...,    0,    1,    0],
       [  48, 5951,    2, ...,    0,    1,    0],
       [  12, 2096,    2, ...,    0,    0,    1],
       ...,
       [  12,  804,    4, ...,    0,    1,    0],
       [  45, 1845,    4, ...,    0,    1,    0],
       [  45, 4576,    3, ...,    0,    1,    0]])

In [188]:
X_train = x
y_train = y

In [192]:
def prepare_dataset():
    #download the german_credit dataset
    urlretrieve('http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.data', 'german.data')
    german_df = pd.read_csv('german.data', delimiter=' ',header=None)
    
    #download the doc file that have the Description of the German credit dataset.
    urlretrieve('http://archive.ics.uci.edu/ml/machine-learning-databases/statlog/german/german.doc', 'german.doc')
    f = open('german.doc')
    
    # add the columns to the dataset
    german_df.columns=['account_bal','duration','payment_status','purpose',
                   'credit_amount','savings_bond_value','employed_since',
                   'intallment_rate','sex_marital','guarantor','residence_since',
                   'most_valuable_asset','age','concurrent_credits','type_of_housing',
                   'number_of_existcr','job','number_of_dependents','telephon',
                   'foreign','target']
    
    # replace some attributes value 
    german_df= german_df.replace(['A11','A12','A13','A14', 'A171','A172','A173','A174','A121','A122','A123','A124'],
                  ['neg_bal','positive_bal','positive_bal','no_acc','unskilled','unskilled','skilled','highly_skilled',
                   'none','car','life_insurance','real_estate'])
    
    # label encoding
    le= LabelEncoder()
    le.fit(german_df.target)
    german_df.target=le.transform(german_df.target)

    # sklearn preprocessing for dealing with categorical variables
    # Create a label encoder object
    le = LabelEncoder()
    le_count = 0

    # Iterate through the columns
    for col in german_df:
        if german_df[col].dtype == 'object':
            # If 2 or fewer unique categories
            if len(list(german_df[col].unique())) <= 2:
                # Train on the training data
                le.fit(german_df[col])
                # Transform both training and testing data
                german_df[col] = le.transform(german_df[col])

                # Keep track of how many columns were label encoded
                le_count += 1

    print('%d columns were label encoded.' % le_count)
    
    # get the data and the label
    x, y = german_df.drop('target', axis=1), german_df['target']
    x = x.to_numpy()
    y = y.to_numpy()
    
    # split to train and test
    # x_train, x_test, y_train, y_test= train_test_split(x,y, test_size=.2, random_state=42)
    
    # safe the train and test file 
    # np.save(f'german_credit_train.npy', {'X': x_train, 'y': y_train})
    # np.save(f'german_credit_test.npy', {'X': x_test, 'y': y_test})
    

    # scale each feature to 0-1
#     scaler = MinMaxScaler(feature_range = (0, 1))

#     # fit on features dataset
#     scaler.fit(x)
#     x = scaler.transform(x)
#     scaler.fit(x_train)
#     scaler.fit(x_test)
#     x_train= scaler.transform(x_train)
#     x_test= scaler.transform(x_test)
    return x, y

In [195]:
x, y = prepare_dataset()
x

2 columns were label encoded.


array([['neg_bal', 6, 'A34', ..., 1, 1, 0],
       ['positive_bal', 48, 'A32', ..., 1, 0, 0],
       ['no_acc', 12, 'A34', ..., 2, 0, 0],
       ...,
       ['no_acc', 12, 'A32', ..., 1, 0, 0],
       ['neg_bal', 45, 'A32', ..., 1, 1, 0],
       ['positive_bal', 45, 'A34', ..., 1, 0, 0]], dtype=object)

In [189]:
dataset_dict = {
    1 : 'german_credit',
    2 : 'adult_income',
    3 : 'compas',
    4 : 'default_credit',
    5 : 'marketing'
    
}

data_dict = {
    'german_credit'     : ('german_credit','target'),
    'adult_income'      : ('adult_income', 'income'),
    'compas'            : ('compas', 'low_risk'),
    'default_credit'    : ('default_credit', 'good_credit'),
    'marketing'         : ('marketing', 'subscribed')      
}  

space_DNN = {
    'units_l1'      : hp.choice('units_l1', [25, 50, 75, 100, 150, 200]),
    'dropout_l1'    : hp.uniform('dropout_l1', 0, 0.5),
    'units_l2'      : hp.choice('units_l2', [25, 50, 75, 100, 150, 200]),
    'dropout_l2'    : hp.uniform('dropout_l2', 0, 0.5),
    'nbr_layers'    : hp.choice('nbr_layers',
                    [
                        {'layers':'two',              
                        },
                        {'layers':'three',
                        'units_l3_a': hp.choice('units_l3_a', [25, 50, 75, 100, 150, 200]),
                        'dropout_l3_a': hp.uniform('dropout_l3_a', 0, 0.5),     
                        }, 
                        {'layers':'four',
                        'units_l3_b': hp.choice('units_l3_b', [25, 50, 75, 100, 150, 200]),
                        'dropout_l3_b': hp.uniform('dropout_l3_b', 0, 0.5), 
                        'units_l4': hp.choice('units_l4', [25, 50, 75, 100, 150, 200]),
                        'dropout_l4': hp.uniform('dropout_l4', 0, 0.5),     
                        },
                    ]),
    'optimizer'     : hp.choice('optimizer', ['rmsprop', 'adam', 'sgd']),
    'epochs'        : 30,
    }


# DNN
def obj_func__DNN(params, rseed):

#     X_train, y_train= prepare_data(data, rseed)
    model = Sequential()

    # first layer
    model.add(Dense(params['units_l1'], input_dim = X_train.shape[1]))
    model.add(Dropout(params['dropout_l1']))
    model.add(Activation('relu'))
    
    # second layer
    model.add(Dense(params['units_l2']))
    model.add(Dropout(params['dropout_l2']))
    model.add(Activation('relu'))

    if params['nbr_layers']['layers']== 'three':
        # third layer
        model.add(Dense(params['nbr_layers']['units_l3_a']))
        model.add(Dropout(params['nbr_layers']['dropout_l3_a']))
        model.add(Activation('relu'))

    if params['nbr_layers']['layers']== 'four':
        # third layer
        model.add(Dense(params['nbr_layers']['units_l3_b']))
        model.add(Dropout(params['nbr_layers']['dropout_l3_b']))
        model.add(Activation('relu'))
        # fourth layer
        model.add(Dense(params['nbr_layers']['units_l4']))
        model.add(Dropout(params['nbr_layers']['dropout_l4']))
        model.add(Activation('relu'))

    # final layers
    model.add(Dense(1))
    model.add(Activation('sigmoid'))

    # loss  function and optimizers
    model.compile(loss='binary_crossentropy', metrics=['accuracy'], optimizer=params['optimizer'])

    result = model.fit(X_train, y_train,
              epochs=params['epochs'],
              verbose=0,
              validation_split=0.1)

    #get the highest validation accuracy of the training epochs
    validation_acc = np.mean(result.history['val_accuracy']) 
    print('Best validation acc of epoch:', validation_acc)
    return {'loss': -validation_acc, 'status': STATUS_OK, 'model': model}





if __name__ == '__main__':

#     # parser initialization
#     parser = argparse.ArgumentParser(description='Script pretraining DNN models')
#     parser.add_argument('--dataset', type=str, default='german_credit', help='german_credit,adult_income, compas, default_credit, marketing')
#     parser.add_argument('--rseed', type=int, default=0, help='random seed: choose between 0 - 9')
#     parser.add_argument('--model_class', type=str, default='DNN', help='DNN, RF, AdaBoost, XgBoost')
#     parser.add_argument('--nbr_evals', type=int, default=25, help='Number of evaluations for hyperopt')

#     # get input
#     args = parser.parse_args()
#     dataset = args.dataset
#     rseed = args.rseed
#     model_class = args.model_class
#     nbr_evals = args.nbr_evals
    rseed = 0
    dataset = 'german_credit'
    nbr_evals = 25
    # create directory to save outputs
    outdir = './pretrained/{}/'.format(dataset)

    if not os.path.exists(outdir):
        os.mkdir(outdir)
    
    
    # filenames of saved objects
    model_name = '{}{}_{}.h5'.format(outdir, model_class, rseed)
    stats_name = '{}{}_{}.txt'.format(outdir, model_class, rseed)

    
    # Initialize an empty trials database
    trials = Trials()
    print(X_train.dtype)
    # Perform the evaluations on the search space
    obj_func__DNN = partial(obj_func__DNN, rseed=rseed)
    best = fmin(obj_func__DNN, space_DNN, algo=tpe.suggest, trials=trials, max_evals=nbr_evals)

    # get params of the best model
    best_params = space_eval(space_DNN, best)
    # get the best model
    best_model = trials.best_trial['result']['model']

    # accuracy of the best model
#     X_train, y_train = prepare_data(dataset, rseed)
    acc_train = best_model.evaluate(X_train, y_train)[1]
    print(acc_train)
#         acc_test = best_model.evaluate(X_test, y_test)[1]
#         acc_sg = best_model.evaluate(X_sg, y_sg)[1]

#         save the best model as bbox
    best_model.save(model_name)



    # save best models params and perfs
    with open(stats_name,'w') as myFile:
        myFile.write('Accuracy train: {}\n'.format(acc_train))
#             myFile.write('Accuracy test: {}\n'.format(acc_test))
#             myFile.write('Accuracy sg: {}\n'.format(acc_sg))
        myFile.write('Model params: {}\n'.format(best_params))


int64
Best validation acc of epoch:                         
0.6813333392143249                                    
Best validation acc of epoch:                                                    
0.6800000071525574                                                               
Best validation acc of epoch:                                                    
0.6800000071525574                                                               
Best validation acc of epoch:                                                    
0.6800000071525574                                                               
Best validation acc of epoch:                                                    
0.6800000071525574                                                               
Best validation acc of epoch:                                                    
0.6833333373069763                                                               
Best validation acc of epoch:                                   

In [61]:
x_train

array([[  60, 6836,    3, ...,    0,    1,    0],
       [  21, 2319,    2, ...,    0,    1,    0],
       [   6, 1236,    2, ...,    0,    1,    0],
       ...,
       [  24, 5804,    4, ...,    0,    1,    0],
       [  12, 1484,    2, ...,    0,    1,    0],
       [   6,  932,    3, ...,    0,    1,    0]])